# Analysing Wikipedia Pages

## Introduction

In this project, we will implement the MapReduce framework to parallel process chunks of data scraped from [Wikipedia](https://www.wikipedia.org/). Our dataset contains 54 megabytes worth of articles that cover a range of topics. The articles have been saved in raw HTML format using the last component of their URLs as the filename (this component directs to the specific article). They are all located in the `wiki` folder.

Our main goals will be the following:

* Search for all occurences of a string in all of the files.
* Provide a case-insensitive option to the search.
* Refine the result by providing the specific locations of the files that contain the string

Let's explore the `wiki` folder...


In [1]:
# List all of the files in the wiki folder. We can use the os.path.join() function.

import os

file_names = os.listdir('wiki')
for file_name in file_names:
    print(file_name)


Bay_of_ConcepciC3B3n.html
Bye_My_Boy.html
Valentin_Yanin.html
Kings_XI_Punjab_in_2014.html
William_Harvey_Lillard.html
Radial_Road_3.html
George_Weldrick.html
Zgornji_Otok.html
Blue_Heelers_(season_8).html
Taggen_Nunatak.html
Henri_BraqueniC3A9.html
Vrila.html
William_Henry_Porter.html
Clive_Brown_(footballer).html
Blick_nach_Rechts.html
Central_District_(Rezvanshahr_County).html
Alexios_Aspietes.html
Mei_Lanfang.html
Wangeroogeclass_tug.html
Dowell_Philip_O27Reilly.html
Coalville_Town_railway_station.html
Gennady_Lesun.html
Bartrum_Glacier.html
Victor_S._Mamatey.html
Gottfried_Keller.html
Table_Point_Formation.html
Nobuhiko_Ushiba.html
Master_of_Space_and_Time.html
Early_medieval_states_in_Kazakhstan.html
Eressa_aperiens.html
Myrtle_(sternwheeler).html
Abanycha_bicolor.html
JeecyVea.html
Aubrey_Fair.html
Ingrid_GuimarC3A3es.html
Urban_chicken.html
Elgin_National_Watch_Company.html
AlMidan.html
Antae_temple.html
Metis_Institute_of_Polytechnic.html
Sverre_Solberg.html
John_Reid_(British

In [2]:
# Count and display the number of files in the wiki folder

print(len(file_names))

999


In [3]:
# Read the first file in the wiki folder and print its contents. We need to join the name of the file with the wiki folder using the os.path.join() function.

folder_name = 'wiki'
file_name = file_names[0]
with open(os.path.join(folder_name, file_name), errors='ignore') as f:
    print(f.read())

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Bay of Concepción - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Bay_of_Concepción","wgTitle":"Bay of Concepción","wgCurRevisionId":647460156,"wgRevisionId":647460156,"wgArticleId":16044270,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Coordinates on Wikidata","All stub articles","Landforms of Bío Bío Region","Bays of Chile","Bío Bío Region geography stubs"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNa

We can see that there are 999 files in the `wiki` folder and each contains raw HTML data. For the purposes of this project, we'll just treat them as plain text files.

## Adding the MapReduce Framework

Now we have a feel for the dataset, let's implement a MapReduce framework that we can use throughout the project to manipulate our data. We will need to design specific mapper and reducer functions for each task.

In [4]:
# Create a MapReduce framework

import math
import functools
from multiprocessing import Pool



def make_chunks(data, num_chunks):
    chunk_size = math.ceil(len(data) / num_chunks)
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]


def map_reduce(data, num_processes, mapper, reducer):
    chunks = make_chunks(data, num_processes)
    with Pool(num_processes) as pool:
        chunk_results = pool.map(mapper, chunks)
    return functools.reduce(reducer, chunk_results)

# Use MapReduce to count the total number of lines in all files

def mapper_line_count(files_chunk):
    directory = folder_name
    total_line_count = 0
    for file_name in files_chunk:
        with open(os.path.join(directory, file_name)) as f:
            total_line_count += len([line for line in f.readlines()])
    return total_line_count

def reducer_line_count(count1, count2):
    return count1 + count2


total_lines = map_reduce(file_names, 8, mapper_line_count, reducer_line_count)
print(total_lines)



499797


## Exact Match Search

Next, we will implement a MapReduce algorithm to locate all lines in all files from the wiki folder that contain a given string.

The output will be a dictionary where the keys are the file names and the values are the list of all line numbers that contain the target string.

For our example, we'll locate all occurrences of the string `'data'` in the files stored in the `wiki` folder.

In [5]:
# Create a mapper and reducer function to use with the map_reduce() function created above.

def map_string_match(files_chunk):
    line_dict = {}
    directory = folder_name
    
    for file_name in files_chunk:
        with open(os.path.join(directory, file_name)) as f:
            index = 0
            for line in f.readlines():
                if target in line:
                    if file_name in line_dict:
                        line_dict[file_name].append(index)
                    else:
                        line_dict[file_name] = [index]
                index += 1
                
    return line_dict

def reduce_string_match(dict1, dict2):
    merged_dict = {}
    merged_dict.update(dict1)
    merged_dict.update(dict2)
    return merged_dict

# Let's test our functions.

target = 'data'

target_matches = map_reduce(file_names, 8, map_string_match, reduce_string_match)
print(target_matches)

{'Bay_of_ConcepciC3B3n.html': [6, 45, 58, 60, 62, 105, 188, 205], 'Bye_My_Boy.html': [276, 359, 376], 'Valentin_Yanin.html': [101, 144, 227, 244], 'Kings_XI_Punjab_in_2014.html': [221, 229, 237, 245, 253, 269, 277, 293, 301, 317, 325, 341, 374, 376, 381, 383, 388, 390, 395, 397, 402, 564, 647, 664], 'William_Harvey_Lillard.html': [45, 65, 81, 129, 212, 229], 'Radial_Road_3.html': [52, 103, 301, 505, 588, 605], 'George_Weldrick.html': [194, 277, 294], 'Zgornji_Otok.html': [6, 53, 55, 65, 69, 211, 260, 262, 311, 394, 411], 'Blue_Heelers_(season_8).html': [49, 79, 82, 105, 107, 125, 127, 133, 135, 141, 143, 660, 695, 730, 739, 886, 969, 986], 'Taggen_Nunatak.html': [6, 44, 46, 48, 93, 176, 193], 'Henri_BraqueniC3A9.html': [43, 46, 92, 175, 192], 'Vrila.html': [6, 57, 59, 69, 73, 99, 100, 102, 151, 234, 251], 'William_Henry_Porter.html': [48, 88, 171, 188], 'Clive_Brown_(footballer).html': [146, 229, 246], 'Blick_nach_Rechts.html': [43, 46, 134, 170, 253, 270], 'Central_District_(Rezvansha

Let's verify our results by targeting the first line identified above...

In [6]:
folder_name = 'wiki'
file_name = 'Bay_of_ConcepciC3B3n.html'
with open(os.path.join(folder_name, file_name)) as f:
    lines = [line for line in f.readlines()]
    
print('data' in lines[6])

True


## Case Insensitive Search

Now let's modify the function so that the search is case insensitive. This means that if someone enters the target string using a different case, or if the cases of the strings within the dataset vary, all of the matches will still be returned.

In [7]:
# Make the mapper function case insensitive

def map_string_match(files_chunk):
    line_dict = {}
    directory = folder_name
    
    for file_name in files_chunk:
        with open(os.path.join(directory, file_name)) as f:
            index = 0
            for line in f.readlines():
                if target.lower() in line.lower():
                    if file_name in line_dict:
                        line_dict[file_name].append(index)
                    else:
                        line_dict[file_name] = [index]
                index += 1
                
    return line_dict

# Let's test our case insensitive function by using the same target as before but in upper case.

target = 'DATA'

target_matches_insensitive = map_reduce(file_names, 8, map_string_match, reduce_string_match)
print(target_matches_insensitive)

{'Bay_of_ConcepciC3B3n.html': [6, 45, 58, 60, 62, 105, 188, 205], 'Bye_My_Boy.html': [276, 359, 376], 'Valentin_Yanin.html': [101, 144, 227, 244], 'Kings_XI_Punjab_in_2014.html': [221, 229, 237, 245, 253, 269, 277, 293, 301, 317, 325, 341, 374, 376, 381, 383, 388, 390, 395, 397, 402, 564, 647, 664], 'William_Harvey_Lillard.html': [45, 65, 81, 129, 212, 229], 'Radial_Road_3.html': [52, 103, 301, 505, 588, 605], 'George_Weldrick.html': [194, 277, 294], 'Zgornji_Otok.html': [6, 53, 55, 65, 69, 211, 260, 262, 311, 394, 411], 'Blue_Heelers_(season_8).html': [49, 79, 82, 105, 107, 125, 127, 133, 135, 141, 143, 660, 695, 730, 739, 886, 969, 986], 'Taggen_Nunatak.html': [6, 44, 46, 48, 93, 176, 193], 'Henri_BraqueniC3A9.html': [43, 46, 92, 175, 192], 'Vrila.html': [6, 57, 59, 69, 73, 99, 100, 102, 151, 234, 251], 'William_Henry_Porter.html': [48, 88, 171, 188], 'Clive_Brown_(footballer).html': [146, 229, 246], 'Blick_nach_Rechts.html': [43, 46, 134, 170, 253, 270], 'Central_District_(Rezvansha

## Checking the Implementation

Let's further verify that the new implementation works by seeing if it finds more matches than the previous implementation.

In [8]:
# For each file, if the case insensitive search returned more mathces, print the file name along with the number of new matches

additional_matches = {}
for file_name in target_matches_insensitive:
    original_matches = len(target_matches[file_name])
    insensitive_matches = len(target_matches_insensitive[file_name])
    if file_name not in target_matches:
        additional_matches[file_name] = insensitive_matches
    elif insensitive_matches > original_matches:
        additional_matches[file_name] = insensitive_matches - original_matches

for file_name in additional_matches:
    print('Found {} additional matches in file {}'.format(additional_matches[file_name], file_name))

Found 1 additional matches in file Table_Point_Formation.html
Found 1 additional matches in file Ingrid_GuimarC3A3es.html
Found 2 additional matches in file Jules_Verne_ATV.html
Found 1 additional matches in file Pictogram.html
Found 2 additional matches in file Claire_Danes.html
Found 1 additional matches in file PTPRS.html
Found 1 additional matches in file A_Beautiful_Valley.html
Found 1 additional matches in file Mudramothiram.html
Found 2 additional matches in file Gordon_Bau.html
Found 1 additional matches in file Embraer_Unidade_GaviC3A3o_Peixoto_Airport.html
Found 3 additional matches in file Code_page_1023.html
Found 1 additional matches in file Cryptographic_primitive.html
Found 1 additional matches in file Alex_Kurtzman.html
Found 1 additional matches in file Filip_Pyrochta.html
Found 1 additional matches in file Morgana_King.html
Found 1 additional matches in file Don_Parsons_(ice_hockey).html
Found 1 additional matches in file Bias.html
Found 2 additional matches in file T

## Finding Match Positions on Lines

So far, we are only finding the line numbers where there is at least one occurrence. Let's extend the algorithm so that it provides information about the location of the matches in those lines.

The new implementation will return pairs of indices where the first value is the line index and the second index is the index of the first character of the match on that line.
For example `(3, 65)` would indicate that the match is on the thrid line and the string starts at index 65.



In [9]:
# Improve the mapper function to return pairs of line indexes and target indexes within the line.

def map_string_match(files_chunk):
    line_dict = {}
    directory = folder_name
    
    for file_name in files_chunk:
        with open(os.path.join(directory, file_name)) as f:
            
            # Calculate whether target is in line. If so, calculate line length and target length for use in next step
            for line_index, line in enumerate(f.readlines()):
                if target.lower() in line.lower():
                    line_length = len(line)
                    target_length = len(target)
                    
                    # find starting index of each occurence of target within line. Add filenames and line/character index tuples to dictionary.
                    for character_index in range((line_length - target_length) + 1):
                        if line[character_index:character_index + target_length].lower() == target.lower():
                            if file_name in line_dict:
                                line_dict[file_name].append((line_index, character_index))
                            else:
                                line_dict[file_name] = [(line_index, character_index)]
                       
    return line_dict

# Test the improved function 

target = 'Data'

target_matches_final = map_reduce(file_names, 8, map_string_match, reduce_string_match)
print(target_matches_final)


{'Bay_of_ConcepciC3B3n.html': [(6, 422), (45, 628), (45, 650), (58, 447), (58, 692), (60, 18), (62, 568), (62, 590), (105, 40), (105, 748), (105, 789), (105, 814), (188, 1039), (188, 1088), (188, 1132), (205, 125)], 'Bye_My_Boy.html': [(276, 40), (359, 999), (359, 1048), (359, 1092), (376, 125)], 'Valentin_Yanin.html': [(101, 323), (101, 360), (144, 40), (227, 1007), (227, 1056), (227, 1100), (244, 125)], 'Kings_XI_Punjab_in_2014.html': [(221, 487), (221, 510), (229, 487), (229, 510), (237, 485), (237, 508), (245, 449), (245, 472), (253, 451), (253, 474), (269, 485), (269, 508), (277, 449), (277, 472), (293, 451), (293, 474), (301, 451), (301, 474), (317, 485), (317, 508), (325, 451), (325, 474), (341, 449), (341, 472), (374, 459), (374, 482), (376, 498), (376, 521), (381, 465), (381, 488), (383, 525), (383, 547), (388, 492), (388, 515), (390, 446), (390, 469), (395, 501), (395, 523), (397, 447), (397, 470), (402, 423), (402, 446), (564, 40), (647, 1043), (647, 1093), (647, 1137), (664

## Displaying the Results

Our MapReduce algorithms can now find the exact location of all matches. However, with the dictionary it produces, it's not very easy to read.

Let's write the results into a CSV file with four columns:

1. `File`: shows the name of the file of the match.
2. `Line`: shows the index of the line of the match.
3. `Index`: shows the character index of the match within the line.
4. `Context`: shows the text around the match so that users can see the context.

In [10]:
import csv

folder_name = 'wiki'
target = 'Data'

target_matches_final = map_reduce(file_names, 8, map_string_match, reduce_string_match)

context_delta = 30

with open('wikipedia_search_results.csv', mode='w') as file:
    writer = csv.writer(file)
    headers = ['File', 'Line', 'Index', 'Context']
    writer.writerow(headers)
    for file_name in target_matches_final:
        with open(os.path.join(folder_name, file_name)) as file2:
            lines = [line for line in file2.readlines()]
        for match in target_matches_final[file_name]:
            line, index = match[0], match[1]
            row = [file_name, line, index]
            start = index - ((context_delta - len(target)) // 2)
            end = index + ((context_delta - len(target)) // 2)
            row.append(lines[line][start:end])
            writer.writerow(row)

In [11]:
import pandas as pd

df = pd.read_csv('wikipedia_search_results.csv')
print(df.head(10))

                        File  Line  Index                     Context
0  Bay_of_ConcepciC3B3n.html     6    422  nates on Wikidata","All st
1  Bay_of_ConcepciC3B3n.html    45    628  uina.jpg 2x" data-file-wid
2  Bay_of_ConcepciC3B3n.html    45    650  -width="960" data-file-hei
3  Bay_of_ConcepciC3B3n.html    58    447  s, and other data for this
4  Bay_of_ConcepciC3B3n.html    58    692  s, and other data for this
5  Bay_of_ConcepciC3B3n.html    60     18  e class="metadata plainlin
6  Bay_of_ConcepciC3B3n.html    62    568  .svg.png 2x" data-file-wid
7  Bay_of_ConcepciC3B3n.html    62    590  -width="600" data-file-hei
8  Bay_of_ConcepciC3B3n.html   105     40  s="catlinks" data-mw="inte
9  Bay_of_ConcepciC3B3n.html   105    748  nates_on_Wikidata" title="
